In [381]:
import requests
import grequests
from pattern import web
import pandas as pd
import csv
import time
import sys
import re
from ast import literal_eval

In [2]:
# define function to return dom
f = lambda url: web.Element(requests.get(url).text)
g = lambda response: web.Element(response.text)

###Grab teams of each season from 92-93 to 15-16.

In [3]:
url_by_season = \
['http://www.11v11.com/league-tables/premier-league/01-september-'+str(start_year)+'/' for \
                                                            start_year in range(1992, 2016)]
url_by_season

['http://www.11v11.com/league-tables/premier-league/01-september-1992/',
 'http://www.11v11.com/league-tables/premier-league/01-september-1993/',
 'http://www.11v11.com/league-tables/premier-league/01-september-1994/',
 'http://www.11v11.com/league-tables/premier-league/01-september-1995/',
 'http://www.11v11.com/league-tables/premier-league/01-september-1996/',
 'http://www.11v11.com/league-tables/premier-league/01-september-1997/',
 'http://www.11v11.com/league-tables/premier-league/01-september-1998/',
 'http://www.11v11.com/league-tables/premier-league/01-september-1999/',
 'http://www.11v11.com/league-tables/premier-league/01-september-2000/',
 'http://www.11v11.com/league-tables/premier-league/01-september-2001/',
 'http://www.11v11.com/league-tables/premier-league/01-september-2002/',
 'http://www.11v11.com/league-tables/premier-league/01-september-2003/',
 'http://www.11v11.com/league-tables/premier-league/01-september-2004/',
 'http://www.11v11.com/league-tables/premier-league

In [4]:
# for collecting all teams appeared in the 24 seasons
teams_pool = []
teams_by_season = {}

In [5]:
# by each season(24 in total), 0-23
for season in range(24):
    print 'time point: '+url_by_season[season].split('/')[-2] + '\n----------------------'
    dom = f(url_by_season[season])
    num = len(dom.by_tag('tbody')[0].by_tag('tr'))
    teams = [str(dom.by_tag('tbody')[0].by_tag('tr')[ranking].by_tag('td')[1].content.split('/')[2]) \
             for ranking in range(num)]
    # add teams to teams_pool
    for t in teams:
        if t not in teams_pool:
            teams_pool.append(t)
    # teams for each season, start_year
    teams_by_season[range(1992, 2016)[season]] = teams

time point: 01-september-1992
----------------------
time point: 01-september-1993
----------------------
time point: 01-september-1994
----------------------
time point: 01-september-1995
----------------------
time point: 01-september-1996
----------------------
time point: 01-september-1997
----------------------
time point: 01-september-1998
----------------------
time point: 01-september-1999
----------------------
time point: 01-september-2000
----------------------
time point: 01-september-2001
----------------------
time point: 01-september-2002
----------------------
time point: 01-september-2003
----------------------
time point: 01-september-2004
----------------------
time point: 01-september-2005
----------------------
time point: 01-september-2006
----------------------
time point: 01-september-2007
----------------------
time point: 01-september-2008
----------------------
time point: 01-september-2009
----------------------
time point: 01-september-2010
----------------

In [380]:
print teams_pool

['norwich-city', 'blackburn-rovers', 'queens-park-rangers', 'everton', 'leeds-united', 'arsenal', 'coventry-city', 'ipswich-town', 'manchester-city', 'middlesbrough', 'oldham-athletic', 'manchester-united', 'sheffield-wednesday', 'aston-villa', 'chelsea', 'liverpool', 'southampton', 'crystal-palace', 'sheffield-united', 'nottingham-forest', 'tottenham-hotspur', 'wimbledon', 'newcastle-united', 'west-ham-united', 'swindon-town', 'leicester-city', 'bolton-wanderers', 'sunderland', 'derby-county', 'barnsley', 'charlton-athletic', 'watford', 'bradford-city', 'fulham', 'birmingham-city', 'west-bromwich-albion', 'portsmouth', 'wolverhampton-wanderers', 'wigan-athletic', 'reading', 'hull-city', 'stoke-city', 'burnley', 'blackpool', 'swansea-city', 'cardiff-city', 'afc-bournemouth']


In [6]:
print str(len(teams_pool))+' teams have played in the Premier League for the past 24 seasons, see the above list'

47 teams have played in the Premier League for the past 24 seasons, see the above list


In [7]:
# key:start_year, e.g. 1992
teams_by_season[1992]

['norwich-city',
 'blackburn-rovers',
 'queens-park-rangers',
 'everton',
 'leeds-united',
 'arsenal',
 'coventry-city',
 'ipswich-town',
 'manchester-city',
 'middlesbrough',
 'oldham-athletic',
 'manchester-united',
 'sheffield-wednesday',
 'aston-villa',
 'chelsea',
 'liverpool',
 'southampton',
 'crystal-palace',
 'sheffield-united',
 'nottingham-forest',
 'tottenham-hotspur',
 'wimbledon']

In [8]:
print 'start_year:', teams_by_season.keys()

start_year: [1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]


###Grab match data by season

In [46]:
# in case storing replicate matches
match_pool = []

month = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05',\
         'Jun':'06', 'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}

# in this season
for start_year in [2015]:
    # make csv file
    season = str(start_year)+'-'+str(start_year+1)
    print 'season '+season+'\n'
    path = 'data/'+season+'.csv'
    with open(path, 'a') as fp:
        a = csv.writer(fp, delimiter=',')
        data = [['Date','Home','Away','FTHG','FTAG']]
        #data = [[Date,Home,Away,FTHG,FTAG]]
        a.writerows(data)


    # requestes the urls of all teams
    print 'requests the urls of all teams...'
    urls_by_team = ['http://www.11v11.com/teams/'+team+'/tab/matches/season/'+str(start_year+1)+'/comp/1' \
                   for team in teams_by_season[start_year]]
    rs = (grequests.get(u) for u in urls_by_team)
    responses_by_team = grequests.map(rs) #a list of all responses
    print 'done...\n-------------------------------------'

    for k,response_team in enumerate(responses_by_team):
        dom = g(response_team)

        # requests all matches by this team
        print 'team NO.'+str(k+1)+'\n'+'requests the urls of all matches...'
        urls_by_match = ['http://www.11v11.com'+str(match.by_tag('td')[1].content).split('"')[1] \
                         for match in dom.by_tag('tbody')[0].by_tag('tr')]
        rs1 = (grequests.get(u) for u in urls_by_match)
        responses_by_match = grequests.map(rs1) #a list of all responses\
        print 'done...\n-------------------------------------'

        print 'parsing match data...'
        for i,game in enumerate(dom.by_tag('tbody')[0].by_tag('tr')):
            # open this url we can see goal times
            url = str(game.by_tag('td')[1].content).split('"')[1]
            match_index = url.split('-')[-1][:-1]
            # coz we collect by teams, same match between two teams is repeated twice
            if match_index in match_pool:
                continue
            else:
                match_pool.append(match_index)
            # match brief statistics are easy to grab
            _Date = str(game.by_tag('td')[0].content)
            Date = _Date.split(' ')[0]+'/'+month[_Date.split(' ')[1]]+'/'+_Date.split(' ')[2]
            #print Date
            Home,Away = str(game.by_tag('td')[1].by_tag('a')[0].content).split(' v ')
            #print Home+' vs '+Away
            try:
                FTHG,FTAG = (int(g) for g in str(game.by_tag('td')[3].content).split('-'))
                #print '%d:%d' %(FTHG,FTAG)
            except:
                FTHG,FTAG = '',''
            # store goal times (minute, 0 or 1, scorer) in this list
            goal_times = []
            # if match is 0-0, don't open the match url
            if (FTHG,FTAG) == (0,0):#no goal
                pass
            elif (FTHG,FTAG) == ('',''):#not kickoff
                pass
            else:
                # open a single match
                dom1 = g(responses_by_match[i])
                # home goals
                for goal in dom1.by_tag('div.goals')[0].by_tag('div.home')[0].by_tag('tr'):
                    scorer = str(goal.by_tag('td')[0].content)
                    goal_time = int(goal.by_tag('td.time')[0].content)
                    goal_times.append((goal_time,0,scorer))#0 for home goals
                # away goals
                for goal in dom1.by_tag('div.goals')[0].by_tag('div.away')[0].by_tag('tr'):
                    scorer = str(goal.by_tag('td')[0].content)
                    goal_time = int(goal.by_tag('td.time')[0].content)
                    goal_times.append((goal_time,1,scorer))#1 for away goals
                #sort by scoring minute
                goal_times.sort(key=lambda x: x[0])
                #print goal_times

            with open(path, 'a') as fp:
                a = csv.writer(fp, delimiter=',')
                #data = [['Date','Home','Away','FTHG','FTAG']]
                data = [ [Date,Home,Away,FTHG,FTAG]+goal_times ]
                a.writerows(data)
        print 'done\n-------------------------------------'

season 2015-2016

requests the urls of all teams...
done...
-------------------------------------
team NO.1
requests the urls of all matches...
done...
-------------------------------------
parsing match data...
done
-------------------------------------
team NO.2
requests the urls of all matches...
done...
-------------------------------------
parsing match data...
done
-------------------------------------
team NO.3
requests the urls of all matches...
done...
-------------------------------------
parsing match data...
done
-------------------------------------
team NO.4
requests the urls of all matches...
done...
-------------------------------------
parsing match data...
done
-------------------------------------
team NO.5
requests the urls of all matches...
done...
-------------------------------------
parsing match data...
done
-------------------------------------
team NO.6
requests the urls of all matches...
done...
-------------------------------------
parsing match data...
don

###Read csv files into pd.DataFrame

In [474]:
season = '2009-2010'
path = 'data/'+season+'.csv'
with open(path, 'r') as fl:
    content = fl.readlines()

In [475]:
goals = []
for i,match in enumerate(content):
    if i == 0:
        continue
    try:
        goals.append(int(match.split(',')[3]) + int(match.split(',')[4]))
    except:
        goals.append(0)

In [476]:
print 'max goals in a match: ', max(goals)
print "let's just set 12 goal(columns), any match is not likely to score over 12 goals"

max goals in a match:  10
let's just set 12 goal(columns), any match is not likely to score over 12 goals


In [477]:
matches = len(content)-1
print 'total matches in the current season: ', matches

total matches in the current season:  380


In [478]:
# for goal times, (6, 0, Olivier Giroud) represents Giroud scored a goal at the 6th minute of the match
# 0 represents the goal scored by home team, 1 represents the goal scored by away team
df = pd.DataFrame({'Date': ['']*matches, \
                   'Home': ['']*matches, \
                   'Away': ['']*matches, \
                   'FTHG': [0]*matches, \
                   'FTAG': [0]*matches, \
                   '1 goal': ['']*matches, '2 goal': ['']*matches, '3 goal': ['']*matches, \
                   '4 goal': ['']*matches, '5 goal': ['']*matches, '6 goal': ['']*matches, \
                   '7 goal': ['']*matches, \
                   '8 goal': ['']*matches, \
                   '9 goal': ['']*matches, \
                   '10 goal': ['']*matches, \
                   '11 goal': ['']*matches, \
                   '12 goal': ['']*matches \
                  }, \
                  columns=['Date','Home','Away','FTHG','FTAG','1 goal','2 goal','3 goal',\
                          '4 goal','5 goal','6 goal','7 goal','8 goal','9 goal','10 goal','11 goal','12 goal'] \
                  )

with open(path, 'r') as fl:
    next(fl)
    for row,match in enumerate(fl):
        Date = match.split(',')[0]
        Home = match.split(',')[1]
        Away = match.split(',')[2]
        # if match not kickoff, set -1
        try:
            FTHG = int(match.split(',')[3])
            FTAG = int(match.split(',')[4])
        except:
            FTHG,FTAG = -1,-1
        df.loc[row, ['Date','Home','Away','FTHG','FTAG']] = Date,Home,Away,FTHG,FTAG

        # store goal times
        pat = re.compile('\(.+?\)')
        gos = [literal_eval(x.replace('""', '"')) for x in pat.findall(match)]
        l = 12-len(gos);gos+=['']*l
        #print gos
        df.loc[row, ['1 goal','2 goal','3 goal','4 goal','5 goal','6 goal','7 goal','8 goal',\
                     '9 goal','10 goal','11 goal','12 goal']] = gos

In [479]:
# sort matches by date
df.Date = [a.date() for a in pd.to_datetime(df.Date, dayfirst=True)]
df.sort(columns=['Date', 'Home', 'Away'], inplace=True)
df = df.reset_index(drop=True)
df

,Date,Home,Away,FTHG,FTAG,1 goal,2 goal,3 goal,4 goal,5 goal,6 goal,7 goal,8 goal,9 goal,10 goal,11 goal,12 goal
0,2009-08-15,Aston Villa,Wigan Athletic,0,2,"(31, 1, Hugo Rodallega)","(56, 1, Jason Koumas)",,,,,,,,,,
1,2009-08-15,Blackburn Rovers,Manchester City,0,2,"(3, 1, Emmanuel Adebayor)","(90, 1, Stephen Ireland)",,,,,,,,,,
2,2009-08-15,Bolton Wanderers,Sunderland,0,1,"(5, 1, Darren Bent)",,,,,,,,,,,
3,2009-08-15,Chelsea,Hull City,2,1,"(28, 1, Stephen Hunt)","(37, 0, Didier Drogba)","(90, 0, Didier Drogba)",,,,,,,,,
4,2009-08-15,Everton,Arsenal,1,6,"(26, 1, Denilson)","(37, 1, Thomas Vermaelen)","(41, 1, William Gallas)","(48, 1, Cesc Fabregas)","(69, 1, Cesc Fabregas)","(88, 1, Eduardo)","(90, 0, Louis Saha)",,,,,
5,2009-08-15,Portsmouth,Fulham,0,1,"(13, 1, Bobby Zamora)",,,,,,,,,,,
6,2009-08-15,Stoke City,Burnley,2,0,"(19, 0, Ryan Shawcross)","(33, 0, Stephen Jordan)",,,,,,,,,,
7,2009-08-15,Wolverhampton Wanderers,West Ham United,0,2,"(22, 1, Mark Noble)","(69, 1, Matthew Upson)",,,,,,,,,,
8,2009-08-16,Manchester United,Birmingham City,1,0,"(34, 0, Wayne Rooney)",,,,,,,,,,,
9,2009-08-16,Tottenham Hotspur,Liverpool,2,1,"(44, 0, Benoit Assou-Ekotto)","(56, 1, Steven Gerrard)","(59, 0, Sebastien Bassong)",,,,,,,,,


In [ ]:
df.to_csv('data/'+season+'.csv', index=False)

In [481]:
# I decided to change every file to this new format
for season in ['1992-1993', '1993-1994', '1994-1995', '1995-1996', '1996-1997', '1997-1998', \
               '1998-1999', '1999-2000', '2000-2001', '2001-2002', '2002-2003', '2003-2004', \
               '2004-2005', '2005-2006', '2006-2007', '2007-2008', '2008-2009', '2009-2010', \
               '2010-2011', '2011-2012', '2012-2013', '2013-2014', '2014-2015', '2015-2016']:
    path = 'data/'+season+'.csv'
    with open(path, 'r') as fl:
        content = fl.readlines()
    goals = []
    for i,match in enumerate(content):
        if i == 0:
            continue
        try:
            goals.append(int(match.split(',')[3]) + int(match.split(',')[4]))
        except:
            goals.append(0)
    matches = len(content)-1
    # for goal times, (6, 0, Olivier Giroud) represents Giroud scored a goal at the 6th minute of the match
    # 0 represents the goal scored by home team, 1 represents the goal scored by away team
    df = pd.DataFrame({'Date': ['']*matches, \
                       'Home': ['']*matches, \
                       'Away': ['']*matches, \
                       'FTHG': [0]*matches, \
                       'FTAG': [0]*matches, \
                       '1 goal': ['']*matches, '2 goal': ['']*matches, '3 goal': ['']*matches, \
                       '4 goal': ['']*matches, '5 goal': ['']*matches, '6 goal': ['']*matches, \
                       '7 goal': ['']*matches, \
                       '8 goal': ['']*matches, \
                       '9 goal': ['']*matches, \
                       '10 goal': ['']*matches, \
                       '11 goal': ['']*matches, \
                       '12 goal': ['']*matches \
                      }, \
                      columns=['Date','Home','Away','FTHG','FTAG','1 goal','2 goal','3 goal',\
                              '4 goal','5 goal','6 goal','7 goal','8 goal','9 goal','10 goal','11 goal','12 goal'] \
                      )

    with open(path, 'r') as fl:
        next(fl)
        for row,match in enumerate(fl):
            Date = match.split(',')[0]
            Home = match.split(',')[1]
            Away = match.split(',')[2]
            # if match not kickoff, set -1
            try:
                FTHG = int(match.split(',')[3])
                FTAG = int(match.split(',')[4])
            except:
                FTHG,FTAG = -1,-1
            df.loc[row, ['Date','Home','Away','FTHG','FTAG']] = Date,Home,Away,FTHG,FTAG

            # store goal times
            pat = re.compile('\(.+?\)')
            gos = [literal_eval(x.replace('""', '"')) for x in pat.findall(match)]
            l = 12-len(gos);gos+=['']*l
            #print gos
            df.loc[row, ['1 goal','2 goal','3 goal','4 goal','5 goal','6 goal','7 goal','8 goal',\
                         '9 goal','10 goal','11 goal','12 goal']] = gos
    # sort matches by date
    df.Date = [a.date() for a in pd.to_datetime(df.Date, dayfirst=True)]
    df.sort(columns=['Date', 'Home', 'Away'], inplace=True)
    df = df.reset_index(drop=True)
    df.to_csv('data/'+season+'.csv', index=False)
    print 'finish season: '+season+'\n'

finish season: 1992-1993

finish season: 1993-1994

finish season: 1994-1995

finish season: 1995-1996

finish season: 1996-1997

finish season: 1997-1998

finish season: 1998-1999

finish season: 1999-2000

finish season: 2000-2001

finish season: 2001-2002

finish season: 2002-2003

finish season: 2003-2004

finish season: 2004-2005

finish season: 2005-2006

finish season: 2006-2007

finish season: 2007-2008

finish season: 2008-2009



Above this line

===================================================================================================================

In [9]:
urls = url_by_season # a list of urls

# A simple task to do to each response object
def do_something(response):
    return g(response).by_class('column columns1')[0].by_tag('h2')[0].content.strip()

rs = (grequests.get(u) for u in urls)
responses = grequests.map(rs)

for response in responses:
    print do_something(response)

Premier League table  after close of play on 01 September 1992
Premier League table  after close of play on 01 September 1993
Premier League table  after close of play on 01 September 1994
Premier League table  after close of play on 01 September 1995
Premier League table  after close of play on 01 September 1996
Premier League table  after close of play on 01 September 1997
Premier League table  after close of play on 01 September 1998
Premier League table  after close of play on 01 September 1999
Premier League table  after close of play on 01 September 2000
Premier League table  after close of play on 01 September 2001
Premier League table  after close of play on 01 September 2002
Premier League table  after close of play on 01 September 2003
Premier League table  after close of play on 01 September 2004
Premier League table  after close of play on 01 September 2005
Premier League table  after close of play on 01 September 2006
Premier League table  after close of play on 01 Septemb

In [48]:
import time
import sys

for i in range(15):
    time.sleep(0.1)
    sys.stdout.write("=")
    sys.stdout.flush()

In [47]:
# define function to count runtime of program
def progress():
    for i in range(30):
        time.sleep(0.1)
        sys.stdout.write("\r%d sec" % (i/10) )
        sys.stdout.flush()
progress()

2 sec